In [1]:
import os
os.chdir("../")
%pwd

'c:\\Users\\HP\\Desktop\\ML\\Customer_Churn_Predictor'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [3]:
from ChurnPredictor.constants import *
from ChurnPredictor.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [5]:
import os
import pandas as pd
import numpy as np
from mlProject import logger
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [6]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def data_manipulation(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.drop(['customerID'], axis = 1)
        df['TotalCharges'] = pd.to_numeric(df.TotalCharges, errors='coerce')

        df.drop(labels=df[df['tenure'] == 0].index, axis=0, inplace=True)
        df["SeniorCitizen"]= df["SeniorCitizen"].map({0: "No", 1: "Yes"})

        return df
    
    def object_to_int(self, df: pd.DataFrame) -> pd.DataFrame:
        if df.dtype=='object':
            df = LabelEncoder().fit_transform(df)
        
        return df

    def data_transformation(self):
        data = pd.read_csv(self.config.data_path)

        data = self.data_manipulation(data)
        logger.info("Data Manipulation Done")

        data = data.apply(lambda x: self.object_to_int(x))
        logger.info("Converted from object to int type")

        X = data.drop(columns = ['Churn'])
        y = data['Churn'].values

        X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.30, random_state = 40, stratify=y)

        smote = SMOTE(random_state=42)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
        logger.info("Resampling Done")

        num_cols = ["tenure", 'MonthlyCharges', 'TotalCharges']
        df_std = pd.DataFrame(StandardScaler().fit_transform(data[num_cols].astype('float64')),
                       columns=num_cols)
        
        scaler= StandardScaler()

        X_train_resampled[num_cols] = scaler.fit_transform(X_train_resampled[num_cols])
        X_test[num_cols] = scaler.transform(X_test[num_cols])
        logger.info("Scaling completed")

        y_train_new = pd.DataFrame(y_train_resampled, columns=['Churn'])
        train  = pd.concat([X_train_resampled, y_train_new], axis=1)

        X_test['Churn'] = y_test
        test = X_test

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)


In [7]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transform = DataTransformation(config=data_transformation_config)
    data_transform.data_transformation()
except Exception as e:
    raise e

[2024-05-28 10:14:42,775: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-28 10:14:42,780: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-28 10:14:42,783: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-28 10:14:42,784: INFO: common: created directory at: artifacts]
[2024-05-28 10:14:42,785: INFO: common: created directory at: artifacts/data_transformation]
[2024-05-28 10:14:42,818: INFO: 2676230150: Data Manipulation Done]
[2024-05-28 10:14:42,838: INFO: 2676230150: Converted from object to int type]
[2024-05-28 10:14:43,208: INFO: 2676230150: Resampling Done]
[2024-05-28 10:14:43,217: INFO: 2676230150: Scaling completed]
[2024-05-28 10:14:43,292: INFO: 2676230150: Splited data into training and test sets]
[2024-05-28 10:14:43,293: INFO: 2676230150: (7228, 20)]
[2024-05-28 10:14:43,294: INFO: 2676230150: (2110, 20)]
(7228, 20)
(2110, 20)
